In [2]:
%matplotlib inline

In [3]:
from gfcat_utils import *

In [4]:
import numpy as np
from astropy.io import fits as pyfits
from astropy import wcs as pywcs
from astropy.visualization import simple_norm, ZScaleInterval
import imageio
from matplotlib.patches import Circle, Rectangle

In [5]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [7]:
for eclipse in [#8915,7068,3090,9204,13461,14256,21066,32394,41726,
                #5024,
                #12958,
                #13642,15133,27702,38424,
                42161]:
    for band in ['NUV','FUV']:
        b = band[0].lower()+'d'
        e = str(eclipse).zfill(5)
        print(f"Processing e{e} {band}...")
        tmpdir = "/Users/cm/GFCAT/gfcat/tmp/"
        lc_filename = f"/Users/cm/github/gphoton_working/test_data/e{e}/e{e}-{b}-30s-photom-12_8.csv"
        if not os.path.exists(lc_filename):
            continue
        fn = f"/Users/cm/github/gphoton_working/test_data/e{e}/e{e}-{b}-full-rice.fits"
        fn_mov = f"/Users/cm/github/gphoton_working/test_data/e{e}/e{e}-{b}-30s-rice.fits"
        lc = parse_lightcurves(lc_filename)

        x = np.array([source['xcenter'] for source in lc])
        y = np.array([source['ycenter'] for source in lc])
        cps = np.array([np.nanmean(source['cps']) for source in lc])

        variables = screen_gfcat([eclipse],band=band,aper_radius=12.8,photdir="/Users/cm/github/gphoton_working/test_data")
        print(variables)

        if not len(variables):
            continue
        movmap,flagmap,edgemap,wcs,tranges,exptimes=read_image(fn_mov,hdunum=1)
        movmap[np.where(np.isinf(movmap))]=0 # because it pops out with inf values... IDK
        movmap[np.where(movmap<0)]=0

        for source_ix in variables[eclipse]:        
            assert len(lc[source_ix]['cps'])==np.shape(movmap)[0]
            imgx,imgy = lc[source_ix]['xcenter'],lc[source_ix]['ycenter']

            if not os.path.exists(f'{tmpdir}/e{e}/'):
                os.makedirs(f'{tmpdir}/e{e}/')

            imsz = np.shape(movmap[0])

            # crop on the subframe
            boxsz = 200
            x1, x2, y1, y2 = (max(int(imgy - boxsz),0),
                              min(int(imgy + boxsz),imsz[0]),
                              max(int(imgx - boxsz),0),
                              min(int(imgx + boxsz),imsz[1]))

            # crop on the full frame
            x1_,x2_,y1_,y2_ = (max(int(imsz[0]/2-imsz[0]/2),0),
                               min(int(imsz[0]/2+imsz[0]/2),imsz[0]),
                               max(int(imsz[1]/2-imsz[0]/2),0),
                               min(int(imsz[1]/2+imsz[0]/2),imsz[1]))

            gs = gridspec.GridSpec(nrows=4, ncols=6)#, height_ratios=[1, 1, 2])

            # write individual frames of the animated gif
            for i,frame in enumerate(movmap): # probably eliminate the first / last frame, which always has lower exposure
                fig = plt.figure(figsize=(12,9));
                fig.tight_layout()
                ax = fig.add_subplot(gs[:3,:3])
                opacity = (edgemap[i]+flagmap[i])/2
                # M, N, 3 or M, N, 4
                ax.imshow(edgemap[i][x1_:x2_,y1_:y2_],origin="lower",cmap="Reds",alpha=opacity[x1_:x2_,y1_:y2_])
                ax.imshow(flagmap[i][x1_:x2_,y1_:y2_],origin="lower",cmap="Blues",alpha=opacity[x1_:x2_,y1_:y2_])
                ax.imshow(np.stack([ZScaleInterval()(frame[x1_:x2_,y1_:y2_]),
                                     ZScaleInterval()(frame[x1_:x2_,y1_:y2_]),
                                     ZScaleInterval()(frame[x1_:x2_,y1_:y2_]),
                                     1-opacity[x1_:x2_,y1_:y2_]],axis=2),origin="lower")
                ax.set_xticks([])
                ax.set_yticks([])
                rect = Rectangle((y1-y1_, x1-x1_), 2*boxsz, 2*boxsz, linewidth=1, edgecolor='y', facecolor='none',ls='solid')
                ax.add_patch(rect)

                ax = fig.add_subplot(gs[:3,3:])
                ax.imshow(edgemap[i][x1:x2,y1:y2],origin="lower",cmap="Reds",alpha=opacity[x1:x2,y1:y2])
                ax.imshow(flagmap[i][x1:x2,y1:y2],origin="lower",cmap="Blues",alpha=opacity[x1:x2,y1:y2])
                ax.imshow(np.stack([ZScaleInterval()(frame[x1:x2,y1:y2]),
                                     ZScaleInterval()(frame[x1:x2,y1:y2]),
                                     ZScaleInterval()(frame[x1:x2,y1:y2]),
                                     1-opacity[x1:x2,y1:y2]],axis=2),origin="lower")
                ax.set_xticks([])
                ax.set_xticks([])
                ax.set_yticks([])
                circ = Circle((boxsz,boxsz),20,linewidth=1,edgecolor='y',facecolor='none',ls='solid')
                ax.add_patch(circ)

                ax = fig.add_subplot(gs[3:,:])
                ax.set_xticks([])
                t = np.arange(len(lc[source_ix]['cps']))
                cps = lc[source_ix]['cps']
                cps_err = lc[source_ix]['cps_err']
                min_i,max_i=np.argmin(cps),np.argmax(cps)
                ax.vlines(t[i],cps[min_i]-3*cps_err[min_i],
                               cps[max_i]+3*cps_err[max_i],ls='dotted')
                ax.scatter(t[i],cps[i],c='y',s=100,marker='o')
                ax.errorbar(t,cps,yerr=cps_err*3,fmt='k.-')

                plt.savefig(f'{tmpdir}/e{e}/e{e}-{b}-30s-{str(i).zfill(2)}-{str(source_ix).zfill(5)}.png',dpi=100)
                plt.close('all')

            cleanup=True
            n_frames = np.shape(movmap)[0]
            # write the animated gif
            gif_fn = f'{tmpdir}/e{e}/e{e}-{b}-30s-{str(source_ix).zfill(5)}.gif'
            print(f"writing {gif_fn}")
            with imageio.get_writer(gif_fn, mode='I', fps=6) as writer:
                for i in np.arange(n_frames):
                    frame_fn = f'{tmpdir}/e{e}/e{e}-{b}-30s-{str(i).zfill(2)}-{str(source_ix).zfill(5)}.png'
                    image = imageio.imread(frame_fn)
                    writer.append_data(image)
                    if cleanup:
                        os.remove(frame_fn)
                    

Processing e42161 NUV...

100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.32s/it]


{42161: [10307, 13136, 16599]}

writing /Users/cm/GFCAT/gfcat/tmp//e42161/e42161-nd-30s-10307.gif

writing /Users/cm/GFCAT/gfcat/tmp//e42161/e42161-nd-30s-13136.gif

writing /Users/cm/GFCAT/gfcat/tmp//e42161/e42161-nd-30s-16599.gif

Processing e42161 FUV...